# Predicting Molecular Properties
## Can you measure the magnetic interactions between a pair of atoms?
__CHAMPS (CHemistry And Mathematics in Phase Space)__


`url:   `[https://www.kaggle.com/c/champs-scalar-coupling/overview](https://www.kaggle.com/c/champs-scalar-coupling/overview)    
`Date:     23/Jun/2019`    
`Updated:  23/Jun/2019`   
`Author:   Enrique Pérez Herrero`

# Predict Potential Energy

This notebook uses the number of dictinct atoms in a molecule to predict, accurately, the molecular potential energy.    The coefficients and the intercept of the linear regression give a closed formula to derive this feature.

In [ ]:
import warnings
import numpy as np
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

### Parameters

In [ ]:
# Ploting Parameters
FIGSIZE = (10, 6)
sns.set() # Set seaborn style

# Notebook Parameters
FREE_MEMORY = True
SAVE_CSV = True

# kFold Validation Parameters
RANDOM_STATE = 123
N_SPLITS = 3
SHUFFLE = True
VERBOSE = False

# Data Files
OUTPUT_FILE_A = "potential_energy_upd.csv"
OUTPUT_FILE_B = "molecule_train.csv"
OUTPUT_FILE_C = "molecule_test.csv"

## User functions

In [ ]:
# Input data user functions

DATA_PATH = "../input"

def csv_path(dataset="train", data_path=DATA_PATH):
    return "{}/{}.csv".format(data_path, dataset)

def read_data(dataset='train', data_path=DATA_PATH):
    index_col = None
    index_type = ['train', 'test']
    if dataset in index_type:
        index_col = 'id'
    data_path = csv_path(dataset, data_path=data_path)
    return pd.read_csv(data_path, index_col=index_col)

# Read data and arrange Data Frames

In [ ]:
train = read_data("train")
test = read_data("test")

## List all atoms in the same molecule

In [ ]:
# One row per molecule
molecule_train = pd.DataFrame({"molecule_name" : train["molecule_name"].unique()})
molecule_test = pd.DataFrame({"molecule_name" : test["molecule_name"].unique()})
structures = read_data('structures')
atom_list_df = structures.groupby('molecule_name')['atom'].apply(list)
atom_list_df = atom_list_df.to_frame()

In [ ]:
if FREE_MEMORY:
    del train, test

## Merge atoms list to molecule train and test

In [ ]:
molecule_train = pd.merge(molecule_train, atom_list_df, how='left', on='molecule_name')
molecule_test = pd.merge(molecule_test, atom_list_df, how='left', on='molecule_name')

## Count distinct atoms in a molecule
The number of distinct atoms in a molecule are the features to use for `Potential Energy` prediction

In [ ]:
# Count atoms by type
atoms_list = structures.atom.unique().tolist()
print("Distinct atoms in structures data: \n {}".format(atoms_list))

for atom in atoms_list:
    molecule_train['atom_' + atom] = \
        molecule_train['atom'].apply(lambda x: x.count(atom))
    molecule_test['atom_' + atom] = \
        molecule_test['atom'].apply(lambda x: x.count(atom))

## Merge Potential Energy

In [ ]:
potential_energy = read_data("potential_energy")
molecule_train = pd.merge(molecule_train, potential_energy)

In [ ]:
if FREE_MEMORY:
    del potential_energy, structures
    del atom_list_df

The column `atom` can be useful to calculate the molecular weight. This feature is not useful to predict `potential_energy` but it can be interesting for other predictions in the competition.

In [ ]:
# 1 atomic mass unit (amu) corresponds to 1.660539040 × 10−24 gram
ATOM_MASS = {
    "H": 1.00784,
    "C": 12.0107,
    "N": 14.0067,
    "O": 15.9990,
    "F": 18.9984
}

def mol_weight(atom_list):
    """
    Get the molecular weight from a list of atoms
    """
    return sum(map(lambda x: ATOM_MASS[x], atom_list))

In [ ]:
molecule_train["mol_weight"] = molecule_train.atom.apply(lambda x: mol_weight(x))
molecule_test["mol_weight"] = molecule_train.atom.apply(lambda x: mol_weight(x))

In [ ]:
molecule_train.head()

# Fit Linear Model and Validate Results

## Select Features

In [ ]:
id_feature = 'molecule_name'
target_feature = (set(molecule_train) - set(molecule_test)).pop()
selected_features = list(molecule_test)
selected_features.remove(id_feature)
selected_features.remove('atom')
print("Selected Features: \t{}".format(selected_features))
print("Target Feature: \t{}".format(target_feature))
print("Id Feature: \t\t{}".format(id_feature))

## Train and test data frames

In [ ]:
X = molecule_train[selected_features]
y = molecule_train[target_feature]

## Set k-Fold Validation 

In [ ]:
kfold = KFold(n_splits=N_SPLITS,
              random_state=RANDOM_STATE,
              shuffle=SHUFFLE)

In [ ]:
fold = 0
r2_scores = []
mse_scores = []
lin_reg = LinearRegression()

for in_index, oof_index in kfold.split(X, y):
    fold += 1
    print("- Training Fold: ({}/{})".format(fold, N_SPLITS))
    X_in, X_oof = X.loc[in_index], X.loc[oof_index]
    y_in, y_oof = y.loc[in_index], y.loc[oof_index]
    
    lin_reg.fit(X_in, y_in)
    y_pred = lin_reg.predict(X_oof)
    r2 = r2_score(y_oof, y_pred)
    r2_scores.append(r2)
    mse_score = mean_squared_error(y_oof, y_pred)
    mse_scores.append(mse_score) 

print('\nkFold Validation Results:')
print(' * Average Variance Score (R2): \t{:.4f}'.format(np.mean(r2_scores)))
print(' * Average Mean squared error (MSE): \t{:.4f}'.format(np.mean(mse_score)))
    

Explained variance score R2 equal to 1 is perfect prediction

## Plot last fold validation and predicted Potential Energy

In [ ]:
plt.figure(figsize=FIGSIZE)
plt.plot(y_oof, y_pred)
plt.title("Fold {} Prediction".format(fold))
plt.xlabel("Validation Potential Energy")
plt.ylabel("Predicted Potential Energy")
plt.show()

# Predict Potential Energy for test data
Prediction using all available train data 

In [ ]:
lin_reg.fit(X, y)
y_test = lin_reg.predict(molecule_test[selected_features])
molecule_test[target_feature] = y_test
if FREE_MEMORY:
    del X, y

## Concatenate `molecule_train` and `molecule_test` in the same data frame

In [ ]:
potential_energy_upd = pd.concat([molecule_train[[id_feature, target_feature]],
                                  molecule_test[[id_feature, target_feature]]],
                                 ignore_index=True)

In [ ]:
plt.figure(figsize=FIGSIZE)
molecule_train['potential_energy'].plot.kde(figsize=FIGSIZE, legend=True, label="train")
molecule_test['potential_energy'].plot.kde(figsize=FIGSIZE, legend=True, label="test")
plt.title("Predicted and Training Potential Energy Density Plot")
plt.xlabel("Potential Energy")
plt.show()

## Arrange updated potencial energy data frame

In [ ]:
potential_energy_upd = potential_energy_upd.sort_values(id_feature)
potential_energy_upd.reset_index(drop=True, inplace=True)

In [ ]:
potential_energy_upd.head()

## Save data as csv
This file contains training and predicted testing potential energy, and it can be useful to add this feature in the `scalar_coupling_constant` competition.    
__Files__
* "potential_energy_upd.csv": Updated `potential_energy.csv` with predicted values for test data.
* "molecule_train.csv": Molecule features, including potential energy, with molecules found in train data.
* "molecule_test.csv": Molecule features, including potential energy, with molecules found in train data.

In [ ]:
def save_csv(df, file_name):
    """
    Check and save csv data to kernel main folder 
    """
    assert "csv" in file_name, "Bad file extension"
    assert df.notnull().values.any(), "NaN values in data frame"
    print(f"Saving file {file_name}...")
    df.to_csv(file_name, index=False)

if SAVE_CSV:
    save_csv(potential_energy_upd, OUTPUT_FILE_A)
    save_csv(molecule_train, OUTPUT_FILE_B)
    save_csv(molecule_test, OUTPUT_FILE_C)